# Locking exercise

This exercise contains a small finite element code written in **Python** which analyzes locking on
the pure bending problem:

![pure bending example](images/pure_bending.png)

The finite element code is not yet fully complete and requires some self implementation.

## Implementation

In Python it is necessary to import packages that we want to use. Here, we include the package `femtoe` which contains many finite element helper functions that we are going to use. These functions are defined in the directory `femtoe`. Additionally, we will need some vector and tensor operations in Python. The package of choice is numpy in this case:

In [ ]:
import femtoe
import numpy as np

In the following, we will not do the implementation of the pure bending problem. We will write a function, that generates the mesh, evaluates all elements, assembles the global system, solves the system and plots the results. It looks like this:

In [ ]:
def solve_pure_bending_problem(num_ele_y: int, nu: float, num_gp: int, *, show_plots:  bool=True, use_pian_sumihara: bool=False):
    """
    Solves the pure bending problem for a given number of elements in y-direction and Poisson's ratio.

    Parameters
    ----------
    num_ele_y : int
        Number of elements in y-direction.
    
    nu : float
        Poisson's ratio.
    
    num_gp : int
        Number of Gauss points per element.
    
    show_plots : bool
        If True, the results will be plotted.
    
    use_pian_sumihara : bool
        If True, the Pian-Sumihara formulation will be used.
    """
    # create our mesh and the boundary conditions
    problem = femtoe.pure_bending.PureBending(
        width=0.4,
        height=0.1,
        num_ele_x=4 * num_ele_y,
        num_ele_y=num_ele_y,
        youngs_modulus=27e9,
        nu=nu,
    )

    # setup an empty global stiffness matrix
    global_stiffness_matrix = np.zeros((problem.num_dof, problem.num_dof))

    # iterate over all elements
    for cell in problem.cells:
        # evaluate the element stiffness matrix
        if use_pian_sumihara:
            # Ignore this for now (we will use it later)
            stiffness_matrix = compute_element_stiffness_matrix_pian_sumihara(
                problem, problem.nodes[cell], num_gp
            )
        else:
            # As per default, we will end up here
            # Note: This function is not yet implemented, we will implement it later!
            stiffness_matrix = compute_element_stiffness_matrix(
                problem, problem.nodes[cell], num_gp
            )

        # assemble the element matrix into our global matrix
        femtoe.assemble_matrix(
            global_stiffness_matrix, stiffness_matrix, cell, problem.num_dof_per_node
        )

    # now we have the global stiffness matrix

    # we now need the force vector
    global_force_vector = np.zeros(problem.num_dof)

    # Loop over all elements of the Neumann-surface
    for cell in problem.traction.cells:
        force_vector = femtoe.compute_element_force_vector_surface_traction(
            problem, problem.nodes[cell]
        )

        femtoe.assemble_vector(
            global_force_vector, force_vector, cell, problem.num_dof_per_node
        )

    # Now, we need to apply our Dirichlet boundary condition
    dirichlet_dofs = problem.dirichlet_dofs # these dofs are fixed
    free_dofs = np.delete(np.arange(problem.num_dof), dirichlet_dofs) # these are the remaining dofs

    # delete respective colums and rows from our global stiffness matrix and force vector
    global_stiffness_matrix = global_stiffness_matrix[np.ix_(free_dofs, free_dofs)]
    global_force_vector = global_force_vector[free_dofs]

    # solve linear system
    nodal_displacements = np.linalg.solve(global_stiffness_matrix, global_force_vector)

    # nodal_displacements is still incomplete, it only contains the free dofs. Let's add the dirichlet dofs again
    full_displacements = np.zeros(problem.num_dof)
    full_displacements[free_dofs] = nodal_displacements

    # Finally, we want to plot the results
    # typically, we want to scale the displacements so that we can see them (they need to be small in linear finite elements!)
    if show_plots:
        femtoe.plot(
            problem,
            full_displacements,
            scale_displacements=5,
            use_pian_sumihara=use_pian_sumihara,
        )

    # for reference, we also plot the deflection of point P
    node_id_p = np.argwhere(
        np.linalg.norm(problem.nodes - np.array([0.4, 0]), axis=1) < 1e-9
    )[0, 0]

    # return the deflection of point P in y-direction
    return full_displacements[2 * node_id_p + 1]    

## Exercise 1: Implementation of the $\boldsymbol{k}^{(e)}$

In the snippet above, the element stiffness matrix is not yet implemented.

First recall, the element stiffness matrix is approximated with the Gauss quadrature rule as follows

$$
\boldsymbol{k}^{(e)} \approx \sum_{g=1}^{n_{gp}} \boldsymbol{B}_g^{(e)T} \boldsymbol{C} \boldsymbol{B}_g^{(e)} \det{\boldsymbol{J_g^{(e)}}} w_g
$$

_Note:_ Before you start this exercise, make sure that you understand the given variables in the template.

In [ ]:
def compute_element_stiffness_matrix(
    problem: femtoe.pure_bending.PureBending,
    nodal_coordinates: np.ndarray,
    num_gp: int
) -> np.ndarray:
    """
    Compute the element stiffness matrix for a given element.

    Parameters:
    -----------
    problem: femtoe.pure_bending.PureBending
        An object describing the problem (mesh, boundary conditions, material parameters, etc.)

    nodal_coordinates: np.ndarray
        The coordinates of all nodes of the element (num_nodes_per_ele x dim).

    num_gp: int
        The number of Gauss points to use for numerical integration.

    Returns:
    --------
    element_stiffness_matrix: np.ndarray
        The element stiffness matrix for linear elasticity (num_dof_per_element x num_dof_per_element).
    """
    # initialize empty stiffness matrix for integration
    element_stiffness_matrix = np.zeros(
        (problem.num_dof_per_element, problem.num_dof_per_element)
    )

    # constitutive matrix of material
    C = problem.constitutive_matrix

    # loop over Gauss points
    for gauss_weight, gauss_point in femtoe.gauss_integration(problem.cell_type, num_gp):
        # evaluates the shape functions at the Gauss point
        shape_function_derivatives = femtoe.evaluate_shape_function_derivatives(
            problem.cell_type, gauss_point
        )

        # compute jacobian matrix and its determinant (Note, this function will be implemented later)
        jacobian = compute_jacobian(shape_function_derivatives.T, nodal_coordinates)
        det_jacobian = np.linalg.det(jacobian)

        if det_jacobian <= 0:
            raise RuntimeError("Jacobian is not positive")

        # compute B-operator (Note, this function will be implemented later)
        B_op = compute_b_operator(jacobian, shape_function_derivatives.T, problem.num_dof_per_element)

        # compute element stiffness matrix
        element_stiffness_matrix += B_op.T @ C @ B_op * det_jacobian * gauss_weight

    return element_stiffness_matrix

### Implement Jacobian matrix

The first missing part is the Jacobian matrix evaluated at each Gauß point, namely $\boldsymbol{J}_g^{(e)}$. The Jacobian is defined as

$$
\boldsymbol{J}^{(e)} = \nabla_{\boldsymbol{\xi}} \boldsymbol{x}^{(e)} = \nabla_{\boldsymbol{\xi}} \boldsymbol{N} \hat{\boldsymbol{x}}^{(e)},
$$
which is in index notation
$$
J_{ij}^{(e)} = \frac{\partial N^k}{\partial \xi_i} \hat{x_j^k}
$$
and in Matrix notation
$$
\boldsymbol{J}^{(e)} = \left[\begin{matrix}
  \frac{\partial N^1}{\partial \xi^1} & \cdots & \frac{\partial N^n}{\partial \xi^1} \\
  \frac{\partial N^1}{\partial \xi^2} & \cdots & \frac{\partial N^n}{\partial \xi^2} \\
\end{matrix}\right]_g \left[\begin{matrix}
    \hat{x}_1^1 & \hat{x}_2^1\\
    \vdots & \vdots \\
    \hat{x}_1^n & \hat{x}_2^n\\
\end{matrix}\right].
$$

Compute the Jacobian $\boldsymbol{J}^{(e)}$ using the equation above in the snippet below.

In [ ]:
def compute_jacobian(shape_function_derivatives: np.ndarray, nodal_coordinates: np.ndarray) -> np.ndarray:
    """
    Compute and return the jacobian matrix of the element.

    Parameters:
    -----------
    shape_function_derivatives: np.ndarray
        The derivative of the shape functions with respect to the parameter coordinates (dim x num_dof_per_element).

    nodal_coordinates: np.ndarray
        The coordinates of all nodes of the element (num_nodes_per_ele x dim).

    Returns:
    --------
    np.ndarray
        The jacobian matrix of the element (dim x dim).
    """

    # Note: With numpy, the matrix-matrix multiplication is done with the @ operator, i.e. A @ B is the matrix-matrix product of A and B

    # add computation of jacobian here
    jacobian = np.zeros((nodal_coordinates.shape[0], nodal_coordinates.shape[0]))

    return jacobian

### Implement B-operator

The other missing part is the computation of the B-operator $\boldsymbol{B}_g^{(e)}$ at each Gauß point. In 2D, this matrix reads

$$
\boldsymbol{B}_g^{(e)} = \left[\begin{matrix}
  \frac{\partial N^{1}}{\partial x_1} & 0 & \cdots & \frac{\partial N^{n}}{\partial x_1} & 0 \\
  0 & \frac{\partial N^{1}}{\partial x_2} & \cdots & 0 & \frac{\partial N^{n}}{\partial x_2} \\
  \frac{\partial N^{1}}{\partial x_2} & \frac{\partial N^{1}}{\partial x_1} & \cdots & \frac{\partial N^{n}}{\partial x_2} & \frac{\partial N^{n}}{\partial x_1} \\
\end{matrix}\right].
$$
Compute the B-operator $\boldsymbol{B}_g^{(e)}$ in the snippet below. Remember that the shape function derivatives with respect to the physical coordinates $\boldsymbol{x}$ are computed using the chain rule:
$$
\left[\begin{matrix}
  \frac{\partial N^{1}}{\partial x_1} \cdots \frac{\partial N^{n}}{\partial x_1} \\
  \frac{\partial N^{1}}{\partial x_2} \cdots \frac{\partial N^{n}}{\partial x_2}
\end{matrix}\right]_g = (\boldsymbol{J}^{(e)})^{-1} \left[\begin{matrix}
  \frac{\partial N^{1}}{\partial \xi_1} \cdots \frac{\partial N^{n}}{\partial \xi_1} \\
  \frac{\partial N^{1}}{\partial \xi_2} \cdots \frac{\partial N^{n}}{\partial \xi_2}
\end{matrix}\right]_g.
$$

In [ ]:
def compute_b_operator(jacobian: np.ndarray, shape_function_derivatives: np.ndarray, num_dof_per_element: int) -> np.ndarray:
    """
    Compute and return the B-operator for linear elasticity.

    Parameters:
    -----------
    jacobian: np.ndarray
        Contains the jacobian matrix (dim x dim) as computed by compute_jacobian(...).

    shape_function_derivatives: np.ndarray
        The derivative of the shape functions with respect to the parameter coordinates (dim x num_dof_per_element).

    Returns:
    --------
    np.ndarray
        The B-operator for linear elasticity (num_strain x num_dof_per_element).
    """
    B_op = np.zeros((3, num_dof_per_element))

    # Note: Similar to MATLAB, you can access a subset of a matrix with the syntax A[start_row:end_row:step, start_col:end_col:step]
    # Note also: In Python, the first index is 0, i.e. the first row/column is indexed with 0

    # Functions that you might need:
    # - np.linalg.inv(A): Inverts the matrix A
    # - np.linalg.solve(A, b): Solves the linear system Ax = b (b can also be a matrix)


    return B_op

### Verification

In the next step, we want to verify your implementation. Here, we compare the deflection of point P with a reference variable.

In [ ]:
if abs(solve_pure_bending_problem(num_ele_y=2, nu=1/3, num_gp=4, show_plots=False)-(-0.004464934714642235)) < 1e-9:
    print("It looks like your implementation is correct! 🥳🥳🥳")
else:
    print("Unfortunately, your implementation still contains an error!😭 Please fix it before continuing")

## Exercise 2: Analysis

In the following we will study locking phenomena for the beam example above. Note that the structure is statically determinate (iso-static) and, therefore, the stresses can be determined.

In the analytical solution, $\sigma_{xx}$ is equal to the applied traction and $\sigma_{yy}=\sigma_{xy}=0$. Thus, we can compare the exact stresses with the stress computed with our finite element code.

In the exercise, we are going to study how the results behave when changing the following variables:

* `nu` defines Poisson's ratio $\nu$.
* `num_ele_y` defines how many elements are used in vertical direction of the beam for its discretization.
* `num_gp` sets how many Gauß-points are used within the quadrilateral.

### Task 1: $\nu=0$

Run the code selecting `num_ele_y=1` for the discretization of the beam, `num_gp=4` Gauß points in each direction for the numerical integration, a Poisson's ratio `nu=0`. Are the results consistent with the theoretical description of the stresses? Which is the numerical phenomenon responsible for the differences?

In [ ]:
deflection = solve_pure_bending_problem(num_ele_y=1, nu=0, num_gp=4)

## Task 2: $\nu=\frac{1}{3}$

Run the code again with `nu=1/3` (without varying the other variables). Which is the main difference when considering `nu=0` as compared with `nu=1/3`? Which numerical phenomenon is responsible for these differences?

In [ ]:
deflection = solve_pure_bending_problem(num_ele_y=1, nu=1/3, num_gp=4)

### Task 3: Convergence study

Now, we want to do a convergence study with `num_gp=4`, `nu=1/3` and for three different values of `num_ele_y`: `num_ele_y=5`, `num_ele_y=10` and `num_ele_y=20`. Does a very fine mesh reduce the parasitic stresses? Is this approach affordable in practice?

In [ ]:
for num_ele_y in [5, 10, 20]:
    print(f"{num_ele_y=}")
    deflection = solve_pure_bending_problem(num_ele_y=num_ele_y, nu=1 / 3, num_gp=4)
    print("")
    print("")

### Task 4: Reduced integration

We are going to explore _reduced numerical integration_ as a remedy to reduce locking. Run the code with `num_ele_y=5`, `nu=1/3` and using only one Gauß point in each direction `num_gp=1`. Can we avoid locking using reduced integration in this example? Which parasitic effect is taking place?

In [ ]:
deflection_ri = solve_pure_bending_problem(num_ele_y=5, nu=1/3, num_gp=1)

### Task 5: Pian-Sumihara element

Now we are going to explore a better element type to avoid locking. Therefore, we implement a Pian-Sumihara element.

_Note:_ The implementation is already complete, you only need to run the code. You might still have a look at the implementation.

In [ ]:
def compute_element_stiffness_matrix_pian_sumihara(
    problem: femtoe.pure_bending.PureBending,
    nodal_coordinates: np.ndarray,
    num_gp: int
) -> np.ndarray:
    # setup empty Pian-Sumihara matrices
    num_stress_dofs = 5
    G = np.zeros((num_stress_dofs, problem.num_dof_per_element))
    H = np.zeros((num_stress_dofs, num_stress_dofs))

    # constitutive matrix of material
    Cinv = np.linalg.inv(problem.constitutive_matrix)

    # loop over Gauss points
    for gauss_weight, gauss_point in femtoe.gauss_integration(problem.cell_type, num_gp):
        # setup ansatz for stresses
        P = np.array([[1, 0, 0, gauss_point[1], 0], [0, 1, 0, 0, gauss_point[0]], [0, 0, 1, 0, 0]])
        
        # evaluates the shape functions at the Gauss point
        shape_function_derivatives = femtoe.evaluate_shape_function_derivatives(
            problem.cell_type, gauss_point
        )

        # compute jacobian matrix and its determinant
        jacobian = compute_jacobian(shape_function_derivatives.T, nodal_coordinates)
        det_jacobian = np.linalg.det(jacobian)

        if det_jacobian <= 0:
            raise RuntimeError("Jacobian is not positive")

        # compute B-operator
        B_op = compute_b_operator(jacobian, shape_function_derivatives.T, problem.num_dof_per_element)

        # integrate Pian-Sumihara matrices
        G += P.T @ B_op * det_jacobian * gauss_weight
        H += P.T @ Cinv @ P * det_jacobian * gauss_weight

    # return stiffness matrix of Pian Sumihara element
    element_stiffness_matrix = G.T @ np.linalg.inv(H) @ G

    return element_stiffness_matrix

Run the code with `num_ele_y=1`, `nu=1/3` and full integration (`num_gp=4`). Are the results better when using the Pian-Sumihara element as compared to the standard element above?

In [ ]:
deflection_ph = solve_pure_bending_problem(num_ele_y=1, nu=1/3, num_gp=4, use_pian_sumihara=True)